# Tarea 3 (Pandas)

Hora de entrega: Viernes 28 de Mayo, 11:59 pm

Nombrar al archivo de la siguiente manera: Apellido_código.  
Ejemplo:  Solis_20060983

Entregar la tarea en esta plantilla, no usar una adicional a esta. 




- Antes que nada, ¡necesitamos nuestros datos! 
1. Ve a la página de microdatos del [INEI](http://iinei.inei.gob.pe/microdatos/). 
2. Ve a "Consulta por encuestas" y ve a ENAHO metodología ACTUALIZADA. 
3. Elige aquella que dice "Condiciones de Vida y Pobreza - ENAHO" (la que no es panel). 
4. Elige el año 2020, y en período, elige la anual.   
5. Descarga los **archivos CSV** del Código módulo 1 (Características de la vivienda y el hogar) y el módulo 34 (Sumarias - Variables Calculadas). 

Esta tarea está pensada en ser desarrollada con los CSV (no los .dta o .sav, que son los archivos de stata ó spss respectivamente).


La ENAHO es una encuesta muy amplia que está distribuida en varios módulos. En esta tarea veremos cómo podemos utilizar información de varios de estos módulos para sacar datos interesantes. 

1. Lee ambos archivos csv con pandas y asígnales un nombre para que puedas trabajar con ambos como base de datos. (Si es que sale un error, es debido a que nuestras encuestas están en español y tienen caracteres como ˜ de la ñ o el acento. Por ello, a las opciones de lectura del csv, agrega la siguiente opción ``` encoding = "latin-1" ```. 

(No olvides importar pandas 🐼)


En caso tus columnas salgan con el nombre en mayúscula, puedes correr el siguiente comando para cambiarlas a minúsculas:

``` python3 
df.columns= df.columns.str.lower()
``` 
Donde `df` es el nombre del dataframe.   
(En esta clase no ahondamos en el manejo de strings, pero `lower()` es un método de este tipo de variables para convertir el string a minúscula.)

In [1]:
import pandas as pd
import numpy as np

In [2]:
modulo1 = pd.read_csv("737-Modulo01/737-Modulo01/Enaho01-2020-100.csv", sep = "," ,encoding = 'latin-1')
modulo34 = pd.read_csv("737-Modulo34/737-Modulo34/Sumaria-2020.csv", sep = "," ,encoding = 'latin-1')

C:\Users\sandr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (322) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
modulo1.columns = modulo1.columns.str.lower()
modulo34.columns = modulo34.columns.str.lower()

In [4]:
modulo1.head(3)

,año,mes,conglome,vivienda,hogar,ubigeo,dominio,estrato,periodo,tipenc,...,factor07,factor_p,rechazo_razones,nconglome,sub_conglome,codccpp,nomccpp,longitud,latitud,altitud
0,2020,1,5009,12,11,10101,4,4,1,3,...,149.202744,186.594,,7076,0,1,CIUDAD CHACHAPOYAS,-77.86129,-6.225835,2287
1,2020,1,5009,41,11,10101,4,4,1,3,...,149.202744,186.594,,7076,0,1,CIUDAD CHACHAPOYAS,-77.86129,-6.225835,2287
2,2020,1,5009,56,11,10101,4,4,1,3,...,149.202744,186.594,,7076,0,1,CIUDAD CHACHAPOYAS,-77.86129,-6.225835,2287


In [5]:
modulo34.head(3)

,año,mes,conglome,vivienda,hogar,ubigeo,dominio,estrato,mieperho,totmieho,...,linea,pobreza,tipocuestionario,tipoentrevista,factor07,nconglome,sub_conglome,lineav_rpl,lineav,pobrezav
0,2020,1,5009,12,11,10101,4,4,2,2,...,303.520996,3,0,1,149.202744,7076,0,657,560.468506,4
1,2020,1,5009,41,11,10101,4,4,2,2,...,303.520996,3,0,1,149.202744,7076,0,657,553.731506,4
2,2020,1,5009,98,11,10101,4,4,6,6,...,303.520996,3,0,1,149.202744,7076,0,657,547.194031,3


2. - ¿Cuáles son las dimensiones de la sumaria? (o el número de filas y columnas).
   - ¿Cuáles son las dimensiones del módulo de vivienda? (o el número de filas y columnas).

In [6]:
modulo34.shape  #Tiene 34490 filas y 179 columnas

(34490, 179)

In [7]:
modulo1.shape   #Tiene 53423 filas y 331 columnas

(53423, 331)

3. En esta oportunidad no necesitaremos todas las variables de la ENAHO para hacer esta tarea. 
- Quedémonos con las variables ``` CONGLOME VIVIENDA HOGAR MIEPERHO PERCEPHO POBREZA GASHOG2D INGHOG2D ``` de la sumaria. 
- Quedémonos con las variables ``` CONGLOME VIVIENDA HOGAR RESULT NBI1 NBI2 NBI3 NBI4 NBI5 ``` del módulo de vivienda.

 
 Nota: cada una de las variables es:
- MIEPERHO: número de miembros del hogar. 
- PERCEPHO: número de perceptores del hogar (que reciben un ingreso). 
- POBREZA: Pobreza monetaria según la línea de pobreza. 
- GASHOG2D: Gasto bruto anual del hogar. 
- INGHOG2D: Ingreso anual neto total del hogar. 

Necesidades básicas:
- NBI1: vivienda inadecuada.
- NBI2: hacinamiento
- NBI3: vivienda sin servicios higiénicos
- NBI4: hogares con  niños que no asisten a la escuela.
- NBI5: hogares con alta dependencia económica 

CONGLOME, VIVIENDA y HOGAR son los identificadores únicos a nivel de hogar (como un key id).

In [8]:
modulovivienda = modulo1.copy(deep = True)
sumaria = modulo34.copy(deep = True)

In [10]:
sumaria = sumaria[['conglome','vivienda','hogar','mieperho','percepho','pobreza','gashog2d','inghog2d']]
modulovivienda = modulovivienda[['conglome','vivienda','hogar','result','nbi1','nbi2','nbi3','nbi4','nbi5']] 

In [11]:
(sumaria.shape) , (modulovivienda.shape)

((34490, 8), (53423, 9))

4. El módulo vivienda incluye todos los hogares que entraron en el marco muestral, incluso aquellos que no participaron en la ENAHO por diversos motivos. Para eliminar estas viviendas, quedémonos con aquellas cuyo ``` RESULT``` sea igual a 1 ó 2. Almacena este resultado para después.

In [12]:
modulovivienda['result'].value_counts()

1    31189
7    15332
2     3301
5     1855
3     1301
4      445
Name: result, dtype: int64

In [13]:
modulovivienda.shape

(53423, 9)

In [14]:
aux1 = modulovivienda[modulovivienda['result'] == 1] 
aux2 = modulovivienda[modulovivienda['result'] == 2] 
modulovivienda1 = pd.concat([aux1,aux2])

In [15]:
modulovivienda1.shape   # Ahora tiene el mismo número de observaciones que la sumaria

(34490, 9)

In [16]:
modulovivienda1.sort_values('conglome')

,conglome,vivienda,hogar,result,nbi1,nbi2,nbi3,nbi4,nbi5
7400,5002,11,11,1,0,0,0,0,0
7401,5002,23,11,1,0,0,0,0,0
7403,5002,49,11,1,0,0,0,0,0
7405,5002,73,11,1,0,0,0,0,0
7407,5002,97,11,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
49756,20353,30,11,1,0,0,0,0,0
49755,20353,23,11,1,0,0,0,0,0
49754,20353,17,11,1,0,1,0,0,0
49761,20353,62,11,1,0,0,0,0,0


5. Queremos calcular una serie de variables (ó columnas nuevas):  
En sumaria: 
- Queremos el número de miembros dependientes del hogar . Estos son el número de personas que no perciben un ingreso ó la diferencia entre el total de miembros y el total de perceptores. 
- Queremos el ingreso mensual promedio por perceptor del hogar.
- Queremos la diferencia  entre ingreso y gasto del hogar

In [17]:
sumaria.columns

Index(['conglome', 'vivienda', 'hogar', 'mieperho', 'percepho', 'pobreza',
       'gashog2d', 'inghog2d'],
      dtype='object')

In [18]:
sumaria['dependientes'] = sumaria.eval('mieperho - percepho')

In [19]:
sumaria['dependientes'].value_counts()

0     11466
1      8631
2      7288
3      4260
4      1794
5       631
6       261
7       104
8        28
9        14
11        6
10        5
13        1
12        1
Name: dependientes, dtype: int64

In [20]:
sumaria['ingmensualprom'] = sumaria.eval('inghog2d / (12 * percepho)')

In [21]:
sumaria['ahorro'] = sumaria.eval('inghog2d - gashog2d')

In [22]:
sumaria

,conglome,vivienda,hogar,mieperho,percepho,pobreza,gashog2d,inghog2d,dependientes,ingmensualprom,ahorro
0,5009,12,11,2,2,3,25065.796875,37274.425781,0,1553.101074,12208.628906
1,5009,41,11,2,2,3,28650.019531,41967.570312,0,1748.648763,13317.550781
2,5009,98,11,6,4,3,38136.218750,42292.535156,2,881.094482,4156.316406
3,5015,7,11,4,3,3,26812.154297,37866.210938,1,1051.839193,11054.056641
4,5015,19,11,4,3,3,63311.539062,91681.960938,1,2546.721137,28370.421875
...,...,...,...,...,...,...,...,...,...,...,...
34485,20305,31,11,1,1,3,12997.472656,14227.951172,0,1185.662598,1230.478516
34486,20305,52,11,1,1,3,10690.595703,11442.000000,0,953.500000,751.404297
34487,20305,73,11,1,1,3,11576.803711,12847.000000,0,1070.583333,1270.196289
34488,20305,116,11,4,1,3,23779.029297,21009.353516,3,1750.779460,-2769.675781


6. La variable de `pobreza`, en la sumaria, está codificada como integers 1, 2 y 3. Esto corresponde a:  
`1`: hogar pobre extremo  
`2`: hogar pobre no extremo  
`3`: hogar no  pobre  

En base a esta variable, crea dos variables más:
1. Una variable que tenga `'pobre extremo'`, `'pobre no extremo'` y `'no pobre'` en vez  de 1, 2, 3.
2. Una variable que tenga `'pobre'` y `'no pobre'` (ó que englobe a los hogares pobres y pobres no extremos).



In [23]:
sumaria['pobreza'].value_counts()

3    26766
2     6272
1     1452
Name: pobreza, dtype: int64

In [24]:
sumaria['nivel_pobreza'] = sumaria['pobreza']

In [25]:
etiquetas ={1: 'pobre extremo',
            2: 'pobre no extremo',
            3: 'no pobre'}
sumaria.replace({'nivel_pobreza': etiquetas}, inplace = True)

In [26]:
sumaria['nivel_pobreza'].value_counts()

no pobre            26766
pobre no extremo     6272
pobre extremo        1452
Name: nivel_pobreza, dtype: int64

In [27]:
sumaria['es_pobre'] = sumaria['nivel_pobreza']

In [28]:
labels ={'pobre extremo': 'pobre',
         'pobre no extremo': 'pobre',
         'no pobre':'no pobre'}
sumaria.replace({'es_pobre': labels}, inplace = True)

In [29]:
sumaria['es_pobre'].value_counts()

no pobre    26766
pobre        7724
Name: es_pobre, dtype: int64

In [30]:
sumaria[30:45]

,conglome,vivienda,hogar,mieperho,percepho,pobreza,gashog2d,inghog2d,dependientes,ingmensualprom,ahorro,nivel_pobreza,es_pobre
30,5063,88,11,6,4,2,17170.759766,14013.831055,2,291.954814,-3156.928711,pobre no extremo,pobre
31,5091,7,11,2,2,3,36871.125000,34911.250000,0,1454.635417,-1959.875000,no pobre,no pobre
32,5091,22,11,3,2,3,32181.523438,32914.437500,1,1371.434896,732.914062,no pobre,no pobre
33,5091,50,11,3,2,3,9679.635742,7198.293945,1,299.928914,-2481.341797,no pobre,no pobre
34,5091,63,11,4,3,3,20149.693359,22251.097656,1,618.086046,2101.404297,no pobre,no pobre
35,5091,75,11,5,2,3,25884.496094,29084.033203,3,1211.834717,3199.537109,no pobre,no pobre
36,5091,90,11,5,2,3,17455.892578,32657.830078,3,1360.742920,15201.937500,no pobre,no pobre
37,5091,114,11,2,2,3,20443.921875,32610.068359,0,1358.752848,12166.146484,no pobre,no pobre
38,5091,129,11,4,2,3,15895.880859,16760.130859,2,698.338786,864.250000,no pobre,no pobre
39,5124,10,11,5,3,3,34358.562500,59165.660156,2,1643.490560,24807.097656,no pobre,no pobre


7. Queremos saber el promedio de ingresos de los hogares (inghog2d) por nivel de pobreza. Haz un `groupby` para averiguar esto. 


(Por ejemplo, todos los hogares pobres extremos tendrán un único promedio, y así con las 2 categorías de pobreza adicionales). 

In [31]:
sumaria[['inghog2d']].groupby(sumaria['nivel_pobreza']).mean().reset_index()

,nivel_pobreza,inghog2d
0,no pobre,32014.094308
1,pobre extremo,9980.110857
2,pobre no extremo,15565.160905


8. ¡Ahora uniremos los datos!  
Pero antes un check de sanidad: Verifica que el número  de filas de ambas bases de datos son las mismas (acuérdate que filtramos las observaciones por `result`).  La ENAHO recolecta información de estos 34,490 hogares a través de todos sus módulos. Para utilizar estos módulos a la vez, realizaremos una "unión" (merge).   

Une ambas bases con un _inner merge_. Recuerda que el id que identifica a cada hogar se compone de ` conglome vivienda hogar`.  

In [32]:
sumaria.shape, modulovivienda1.shape

((34490, 13), (34490, 9))

In [33]:
base_final = pd.merge(sumaria,
                      modulovivienda1,
                      left_on = ['conglome', 'vivienda', 'hogar'],
                      right_on = ['conglome', 'vivienda', 'hogar'],
                      how = 'inner', indicator=True)  

base_final

,conglome,vivienda,hogar,mieperho,percepho,pobreza,gashog2d,inghog2d,dependientes,ingmensualprom,ahorro,nivel_pobreza,es_pobre,result,nbi1,nbi2,nbi3,nbi4,nbi5,_merge
0,5009,12,11,2,2,3,25065.796875,37274.425781,0,1553.101074,12208.628906,no pobre,no pobre,1,0,0,0,0,0,both
1,5009,41,11,2,2,3,28650.019531,41967.570312,0,1748.648763,13317.550781,no pobre,no pobre,1,0,0,0,0,0,both
2,5009,98,11,6,4,3,38136.218750,42292.535156,2,881.094482,4156.316406,no pobre,no pobre,1,0,0,0,0,0,both
3,5015,7,11,4,3,3,26812.154297,37866.210938,1,1051.839193,11054.056641,no pobre,no pobre,1,0,0,0,0,0,both
4,5015,19,11,4,3,3,63311.539062,91681.960938,1,2546.721137,28370.421875,no pobre,no pobre,1,0,0,0,0,0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34485,20305,31,11,1,1,3,12997.472656,14227.951172,0,1185.662598,1230.478516,no pobre,no pobre,1,0,0,0,0,0,both
34486,20305,52,11,1,1,3,10690.595703,11442.000000,0,953.500000,751.404297,no pobre,no pobre,1,0,0,0,0,0,both
34487,20305,73,11,1,1,3,11576.803711,12847.000000,0,1070.583333,1270.196289,no pobre,no pobre,1,1,0,0,0,0,both
34488,20305,116,11,4,1,3,23779.029297,21009.353516,3,1750.779460,-2769.675781,no pobre,no pobre,1,0,0,0,0,0,both


In [34]:
base_final['_merge'].value_counts()   # Ambas bases se unieron correctamente

both          34490
right_only        0
left_only         0
Name: _merge, dtype: int64

9. Haz un ```groupby``` que agrupe (`agg`) los datos por nivel de pobreza  y la nbi1. Queremos hallar el promedio y la mediana de los ingresos (inghog2d) y los gastos (gashog2d) por el producto cartesiano de las categorías de estas 2 variables - pobreza y nbi 1 -. 

El producto cartesiano es tan sólo todas las posibles combinaciones entre categorías. 
En este caso, tendremos:
- pobre extremo con nbi1
- pobre extremo sin nbi1
- pobre no extremo con nbi1
- pobre no extremo sin nbi1
- no pobre con nbi1
- no pobre sin nbi1

(Este es un  ejemplo de cómo, al cruzar un indicador de pobreza monetaria -pobreza- y pobreza multidimensional-nbis-, podemos seguir observando brechas).

In [35]:
agrupacion = base_final.groupby(['nivel_pobreza','nbi1']).agg(
    {'inghog2d': ['mean', 'median'],
     'gashog2d': ['mean', 'median']}).reset_index()

agrupacion.columns = [''.join(col).strip() for col in agrupacion.columns.values]
agrupacion

,nivel_pobreza,nbi1,inghog2dmean,inghog2dmedian,gashog2dmean,gashog2dmedian
0,no pobre,0,32760.041129,23489.980469,27007.277047,22380.583984
1,no pobre,1,19440.486354,16117.046875,18074.743293,15901.835938
2,pobre extremo,0,9906.536155,8602.481445,7992.623172,7514.380859
3,pobre extremo,1,10618.739268,9501.479980,8556.479213,7883.616455
4,pobre no extremo,0,15626.399573,11971.979492,13406.471448,12256.365234
5,pobre no extremo,1,15045.327072,12720.994141,13271.803297,12754.585938


10. Crea una variable que nos informe sobre los hogares que tienen las 5 nbis. Esta tomará el valor de 1 si tiene las 5 nbis, 0 en caso contrario (así  tenga 4, 3). 

Calcula el promedio de miembros dependientes por cada categoría de esta nueva variable. 

In [36]:
tipo_datos = {'nbi1': 'int32', 'nbi2': 'int32', 'nbi3': 'int32', 'nbi4': 'int32', 'nbi5': 'int32'}

In [37]:
base_final = base_final.astype(tipo_datos)

In [71]:
base_final['nbi_suma'] = base_final.eval('nbi1 + nbi2 + nbi3 + nbi4 + nbi5')

In [72]:
base_final['nbi_suma'].value_counts()

0    28470
1     5032
2      865
3      112
4       11
Name: nbi_suma, dtype: int64

In [41]:
base_final['nbi_total'] = base_final['nbi_suma']

In [42]:
valores ={1: 0,
         2: 0,
         3: 0,
         4: 0,
         5: 1}
base_final.replace({'nbi_total': valores}, inplace = True)

In [54]:
base_final[['dependientes']].groupby(base_final['nbi_total']).mean().reset_index()

,nbi_total,dependientes
0,0,1.423688


11. Crédito extra: 

Guarda el dataframe del resultado del ejercicio 9 como un csv. (No es necesario adjuntar este csv, con  que el bloque haya corrido está bien). 

In [53]:
agrupacion.to_csv('pobrezamonetaria_pobrezamultidimensional.csv', index = False)

**Tarea terminada**    ┗┃・ ■ ・┃┛ 

(╯°□°）╯︵ ┻━┻